In [210]:
import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.merge(pd.read_csv('train_values.csv'),
                 pd.read_csv('train_labels.csv'))
test = pd.read_csv('test_values.csv')
sample_submission = pd.read_csv('submission_format.csv')

train.shape, test.shape


((260601, 40), (86868, 39))

In [211]:
#train, validation = train_test_split(train, train_size=0.80, test_size=0.20, stratify=train['damage_grade'], random_state=1337)

#train.shape, validation.shape, test.shape



In [212]:
train['damage_grade'].value_counts(normalize=True)

2    0.568912
3    0.334680
1    0.096408
Name: damage_grade, dtype: float64

In [213]:
def wrangle(X):
    X = X.copy()

    X = X.drop(['building_id'], axis=1)

    # X.iloc[:, 14:25] = X.iloc[:, 14:25].astype(str)
    # X.iloc[:, 27:38] = X.iloc[27:38].astype(str)

    # X[['has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
    #    'has_superstructure_cement_mortar_stone', 'has_superstructure_mud_mortar_brick',
    #    'has_superstructure_cement_mortar_brick', 'has_superstructure_timber', 'has_superstructure_bamboo',
    #    'has_superstructure_rc_non_engineered', 'has_superstructure_rc_engineered', 'has_superstructure_other',
    #    'has_secondary_use', 'has_secondary_use_agriculture', 'has_secondary_use_hotel',
    #    'has_secondary_use_rental', 'has_secondary_use_institution', 'has_secondary_use_school',
    #    'has_secondary_use_industry', 'has_secondary_use_health_post', 'has_secondary_use_gov_office',
    #    'has_secondary_use_use_police', 'has_secondary_use_other']] = X[['has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
    #    'has_superstructure_cement_mortar_stone', 'has_superstructure_mud_mortar_brick',
    #    'has_superstructure_cement_mortar_brick', 'has_superstructure_timber', 'has_superstructure_bamboo',
    #    'has_superstructure_rc_non_engineered', 'has_superstructure_rc_engineered', 'has_superstructure_other',
    #    'has_secondary_use', 'has_secondary_use_agriculture', 'has_secondary_use_hotel',
    #    'has_secondary_use_rental', 'has_secondary_use_institution', 'has_secondary_use_school',
    #    'has_secondary_use_industry', 'has_secondary_use_health_post', 'has_secondary_use_gov_office',
    #    'has_secondary_use_use_police', 'has_secondary_use_other']].astype(str)

    # X.iloc[:, 14:25] = X.iloc[:, 14:25].astype(str)
    # X.iloc[:, 27:38] = X.iloc[27:38].astype(str)

    return X

In [214]:
train = wrangle(train)
#validation = wrangle(validation)
test = wrangle(test)

In [215]:
train.dtypes

geo_level_1_id                             int64
geo_level_2_id                             int64
geo_level_3_id                             int64
count_floors_pre_eq                        int64
age                                        int64
area_percentage                            int64
height_percentage                          int64
land_surface_condition                    object
foundation_type                           object
roof_type                                 object
ground_floor_type                         object
other_floor_type                          object
position                                  object
plan_configuration                        object
has_superstructure_adobe_mud               int64
has_superstructure_mud_mortar_stone        int64
has_superstructure_stone_flag              int64
has_superstructure_cement_mortar_stone     int64
has_superstructure_mud_mortar_brick        int64
has_superstructure_cement_mortar_brick     int64
has_superstructure_t

In [216]:
from sklearn.metrics import accuracy_score

majority_class = train['damage_grade'].mode()[0]
y_pred = [majority_class] * len(train['damage_grade'])

print("Train baseline accuracy is: ", accuracy_score(train['damage_grade'], y_pred))

Train baseline accuracy is:  0.5689118614280068


In [217]:
target = 'damage_grade'

train_features = train.drop([target], axis=1)
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

categorical_features = train_features.select_dtypes(exclude='number').nunique().index.tolist()


features = numeric_features + categorical_features

In [218]:
# import category_encoders as ce
#
# encoder = ce.OneHotEncoder()
# X_train_encoded = encoder.fit_transform(train[categorical_features])
# X_test_encoded = encoder.transform(test[categorical_features])
#
# X_train_numeric = train[numeric_features]
# X_test_numeric = test[numeric_features]
#
# X_train_nn = pd.concat([X_train_encoded, X_test_numeric], axis=1)
# X_test_nn = pd.concat([X_test_encoded, X_test_numeric], axis=1)

In [219]:
from sklearn.preprocessing import LabelEncoder


y_train = train[target]
X_train = train[features]

# le = LabelEncoder()
# y_train = le.fit_transform(y_train)

X_test = test[features]

In [220]:
# from sklearn.preprocessing import MinMaxScaler
# from tensorflow.keras.utils import to_categorical
#
# scaler = MinMaxScaler()
#
# X_train_scaled = scaler.fit_transform(X_train_encoded)
# X_test_scaled = scaler.transform(X_test_encoded)
#
# y_train_cat = to_categorical(y_train)

In [221]:
X_train.shape

(260601, 38)

In [222]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler
import category_encoders as ce
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from xgboost import XGBClassifier
from imblearn.over_sampling import RandomOverSampler, SMOTE
from skopt import BayesSearchCV
from sklearn.model_selection import GridSearchCV, cross_val_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

#When hyperparamater tuning, set tune to "True", and mark each model that we want to tune to "True".
tune = False
forest = False
xgboost = False
extratrees = False
neuralnetwork = False

forest_distributions = {
    'model__n_estimators': [50, 100, 150, 200, 250, 300],
    'model__max_depth': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
    'model__max_features': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30,
                            31, 32, 33, 34, 35, 36, 37, 38],
    'model__min_samples_leaf': [2, 3, 4, 5, 6, 7, 8],
    'model__criterion': ['gini', 'entropy'],
    'model__class_weight': ['balanced', 'balanced_subsample']
}

# xgboost_distributions = {
#     'model__n_estimators': [125, 150, 175, 200, 225, 250, 260, 275, 300],
#     'model__max_depth': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
#     'model__learning_rate': [0.03, 0.04, 0.05, 0.07, 0.09, 0.10, 0.12, 0.14, 0.16],
#     'model__min_child_leaf':[1, 2, 3, 4, 5],
#     'model__min_child_weight': [1, 2, 3, 4, 5],
#     'model__colsample_bytree':[0.1, 0.15, 0.2, 0.3, 0.4, 0.50],
#     'model__subsample':[0.6, 0.7, 0.8, 0.9, 1],
#     'model__gamma':[0, 1, 2, 3, 4, 5],
#     'model__scale_pos_weight': [20, 25, 30, 35, 40, 45, 50]
#  }

xgboost_distributions = {
    'model__n_estimators': [270, 275, 280],
    'model__max_depth': [26],
    'model__learning_rate': [0.08],
    'model__min_child_leaf':[2, 3],
    'model__min_child_weight': [1, 2],
    'model__colsample_bytree':[0.19, 0.2, 0.21],
    'model__subsample':[0.93, 0.94, 0.95, 0.96],
    'model__gamma':[0.9, 1, 1.1],
    'model__scale_pos_weight': [35, 36, 37, 38]
 }

extratrees_distributions = {
    'model__n_estimators': [50, 100, 150, 200, 250, 300, 325, 350, 375, 400, 425],
    'model__max_depth': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
    'model__min_samples_leaf': [2, 3, 4, 5, 6, 7, 8],
    'model__max_features': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30,
                            31, 32, 33, 34, 35, 36, 37, 38],
    'model__criterion': ['gini', 'entropy'],
    'model__bootstrap': [True, False],
    'model__class_weight': ['balanced', 'balanced_subsample']
}

# forest_distributions = {
#     'model__n_estimators': [350, 375, 400, 425],
#     'model__max_depth': [29, 30, 31, 32, 33],
#     'model__max_features': [24, 25, 26, 27],
#     'model__min_samples_leaf': [4]
# }

# xgboost_distributions = {
#     'model__n_estimators': [260, 265, 270],
#     'model__max_depth': [26, 27],
#     'model__learning_rate': [0.08, 0.09],
#     'model__min_child_leaf':[3],
#     'model__min_child_weight': [4],
#     'model__colsample_bytree':[0.2],
#     'model__subsample':[0.9, 1],
#     'model__gamma':[0],
#     'model__scale_pos_weight': [25]
#   }

if tune: #If we are hyperparamater tuning, pass no parameters into estimators and find paramaters via GridSearchCV / RandomizedSearchCV

    forest_pipeline = Pipeline([('encoder', ce.OrdinalEncoder()),
                           ('model', RandomForestClassifier(random_state=1337))])

    xgboost_pipeline = Pipeline([('encoder', ce.TargetEncoder()),
                           ('model', XGBClassifier(seed=1337))])

    extratrees_pipeline = Pipeline([('encoder', ce.OrdinalEncoder()),
                                ('model', ExtraTreesClassifier(random_state=1337))])


    forest_search = BayesSearchCV(
        forest_pipeline,
        search_spaces=forest_distributions,
        n_iter=500,
        cv=3,
        scoring='f1_micro',
        random_state=1337,
        n_points=5,
        n_jobs=15
    )
    xgboost_search = BayesSearchCV(
        estimator=xgboost_pipeline,
        search_spaces=xgboost_distributions,
        n_iter=500,
        cv=3,
        scoring='f1_micro',
        random_state=1337,
        n_points=5,
        n_jobs=15
    )
    extratrees_search = BayesSearchCV(
        extratrees_pipeline,
        search_spaces=extratrees_distributions,
        n_iter=500,
        cv=3,
        scoring='f1_micro',
        random_state=1337,
        n_points=5,
        n_jobs=15
    )

    # forest_search = GridSearchCV(
    #     estimator=forest_pipeline,
    #     param_grid=forest_distributions,
    #     cv=3,
    #     verbose=3,
    #     scoring='f1_micro',
    #     n_jobs=15
    # )
    # xgboost_search = GridSearchCV(
    #     estimator=xgboost_pipeline,
    #     param_grid=xgboost_distributions,
    #     cv=3,
    #     verbose=3,
    #     scoring='f1_micro',
    #     n_jobs=15
    # )

    def report_step_forest(optim_result):
        global previous_score
        global i
        global total_i
        score = forest_search.best_score_
        print("Random Forest best score: %s" % score)
        if score > previous_score:
            i = 0
        else:
            i += 1
        print("Iterations since improvement: ", i)
        total_i += 1
        print("Total iterations: ", total_i)
        if i > 6:
            print("Over 6 iterations with no improvement, aborting...")
            return True
        previous_score = score
        print("--------------------------------------")

    def report_step_extratrees(optim_result):
        global previous_score
        global i
        global total_i
        score = extratrees_search.best_score_
        print("Extra trees best score: %s" % score)
        if score > previous_score:
            i = 0
        else:
            i += 1
        print("Iterations since improvement: ", i)
        total_i += 1
        print("Total iterations: ", total_i)
        if i > 6:
            print("Over 6 iterations with no improvement, aborting...")
            return True
        previous_score = score
        print("--------------------------------------")

    def report_step_xgboost(optim_result):
        global previous_score
        global i
        global total_i
        score = xgboost_search.best_score_
        print("XGboost best score: %s" % score)
        if score > previous_score:
            i = 0
        else:
            i += 1
        print("Iterations since improvement: ", i)
        total_i += 1
        print("Total iterations: ", total_i)
        if i > 5:
            print("Over 5 iterations with no improvement, aborting...")
            return True
        previous_score = score
        print("--------------------------------------")

    #X_train, y_train = RandomOverSampler(sampling_strategy='not majority').fit_resample(X_train, y_train)

    if forest:
        i = 0
        total_i = 0
        previous_score = 0

        forest_search.fit(X_train, y_train, callback=report_step_forest)
        forest_train_pred = forest_search.predict(X_train)
        forest_test_pred = forest_search.predict(X_test)

    if xgboost:
        i = 0
        total_i = 0
        previous_score = 0

        xgboost_search.fit(X_train, y_train, callback=report_step_xgboost)
        xgboost_train_pred = xgboost_search.predict(X_train)
        xgboost_test_pred = xgboost_search.predict(X_test)

    if extratrees:
        i = 0
        total_i = 0
        previous_score = 0

        extratrees_search.fit(X_train, y_train, callback=report_step_extratrees)
        extratrees_train_pred_proba = extratrees_search.predict_proba(X_train)
        extratrees_test_pred_proba = extratrees_search.predict_proba(X_test)

    # if neuralnetwork:
    #     def keras_model():
    #         network = Sequential()
    #
    #         network.add(Dense(90, activation='relu'))
    #         network.add(Dense(512, activation='relu'))
    #         network.add(Dropout(0.2))
    #         network.add(Dense(256, activation='relu'))
    #         network.add(Dropout(0.2))
    #         network.add(Dense(128, activation='relu'))
    #         network.add(Dropout(0.1))
    #         network.add(Dense(3, activation='softmax'))
    #
    #         network.compile(optimizer='adam', loss='categorical_crossentropy')
    #
    #         return network
    #
    #     early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    #
    #     # nn.fit(x=X_train_scaled, y=y_train_cat, epochs=9999, validation_split=0.2, callbacks=[early_stop])
    #
    #     nn = KerasClassifier(build_fn=keras_model, epochs=10, batch_size=32,
    #                          validation_split=0.2)

        # nn.fit(X_train_scaled, y_train_cat)

    #When hyperparameter tuning, pass our best estimators into votingclassifier. Only run when all 3 models are being tuned.
    # if forest and xgboost:
    #     voting_model = VotingClassifier(estimators=[('forest', forest_search.best_estimator_), #VotingClassifier is Soft Voting/Majority Rule classifier for unfitted estimators
    #                                                 ('xgboost', xgboost_search.best_estimator_),
    #                                                 ('extratrees', extratrees_search.best_estimator_),],
    #                                     voting='hard') #soft voting per recommendation from sklearn documentation, when used on tuned classifiers.
    #     voting_model_cv = voting_model
    #     voting_model.fit(X_train, y_train)
    #     combined_model_test_pred = voting_model.predict(X_test)

else: #If we are not hyperparameter tuning, pass in our best params(from previous tuning runs).
    forest_pipeline = Pipeline([('encoder', ce.OrdinalEncoder()),
                            ('model', RandomForestClassifier(random_state=1337,
                                                             n_jobs=12, max_depth=30, max_features=25, min_samples_leaf=4,
                                                             n_estimators=350))])

    xgboost_pipeline = Pipeline([('encoder', ce.TargetEncoder()),
                                ('model', XGBClassifier(seed=1337, n_jobs=12, colsample_bytree=0.2, gamma=1,
                                                        learning_rate=0.08, max_depth=26, min_child_leaf=2, min_child_weight=1,
                                                        n_estimators=265, scale_pos_weight=37, subsample=0.94))])

    forest_entropy_pipeline = Pipeline([('encoder', ce.OrdinalEncoder()),
                                ('model', RandomForestClassifier(random_state=1337, n_jobs=12, max_depth=28,
                                                             min_samples_leaf=2, max_features=26, n_estimators=200,
                                                              criterion='entropy'))])


    #X_train, y_train = RandomOverSampler(sampling_strategy='minority').fit_resample(X_train, y_train) #over sample all but the majority class

    forest_pipeline_cv = forest_pipeline
    forest_pipeline.fit(X_train, y_train)
    forest_train_pred = forest_pipeline.predict(X_train)
    forest_test_pred = forest_pipeline.predict(X_test)

    xgboost_pipeline_cv = xgboost_pipeline
    xgboost_pipeline.fit(X_train, y_train)
    xgboost_train_pred = xgboost_pipeline.predict(X_train)
    xgboost_test_pred = xgboost_pipeline.predict(X_test)

    forest_entropy_pipeline_cv = forest_entropy_pipeline
    forest_entropy_pipeline.fit(X_train, y_train)
    forest_entropy__train_pred_proba = forest_entropy_pipeline.predict_proba(X_train)
    forest_entropy_test_pred_proba = forest_entropy_pipeline.predict_proba(X_test)

    #
    # voting_model = VotingClassifier(estimators=[('forest', forest_pipeline),
    #                                             ('xgboost', xgboost_pipeline),
    #                                             ('extratrees', extratrees_pipeline),
    #                                             ('extratrees_entropy', extratrees_entropy_pipeline),
    #                                             ('forest_entropy', forest_entropy_pipeline),],
    #                                 voting='hard', weights=[1, 3, 1, 1, 1], n_jobs=15)
    # voting_model_cv = voting_model
    # voting_model.fit(X_train, y_train)
    # combined_model_test_pred = voting_model.predict(X_test)

[20:36:12] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { min_child_leaf, scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [223]:
from sklearn.metrics import f1_score, classification_report
import numpy as np
if tune:
    if forest:
        print(forest_search.best_params_, '\n')
        print("Best Random Forest CV score: ", forest_search.best_score_, '\n')
        print('\n')

    if xgboost:
        print(xgboost_search.best_params_, '\n')
        print("Best xgboost CV score: ", xgboost_search.best_score_, '\n')
        print('\n')

    if extratrees:
        print(extratrees_search.best_params_, '\n')
        print("Best Extra Trees CV score: ", extratrees_search.best_score_, '\n')
        print('\n')

    # if neuralnetwork:
    #     print("Neural network CV score:", cross_val_score(nn, X_train_scaled, y_train,
    #                                                         scoring='f1_micro', cv=3, n_jobs=1))

    # if forest and xgboost and extratrees:
    #     #print("Voting classifier, final F1-Micro score on validation set: ", f1_score(y_validation, combined_model_pred, average='micro'))
    #     print("Voting classifier, final F1-Micro cross-validation score: ", np.mean(cross_val_score(voting_model_cv, X_train,
    #                                                                                         y_train, scoring='f1_micro',
    #                                                                                         cv=3, n_jobs=15)))
else:

    print("Xgboost F1-Micro: ", np.mean(cross_val_score(xgboost_pipeline_cv, X_train, y_train, scoring='f1_micro', cv=3,
                                                n_jobs=12)), '\n')

    print('\n')

    print("Random forest gini(class balance=None) F1-Micro: ", np.mean(cross_val_score(forest_pipeline_cv, X_train, y_train, scoring='f1_micro', cv=3,
                                                n_jobs=12)), '\n')

    print('\n')

    print("Random forest entropy F1-Micro: ", np.mean(cross_val_score(forest_entropy_pipeline_cv, X_train, y_train, scoring='f1_micro', cv=3,
                                                n_jobs=12)), '\n')

    print('\n')

    #print("Xgboost F1-Micro training score: ", f1_score(y_train, xgboost_train_pred, average='micro'))

    # print("Voting classifier, final voting ensemble cross-validation score: ", np.mean(cross_val_score(voting_model_cv, X_train,
    #                                                                                         y_train, scoring='f1_micro',
    #                                                                                         cv=3, n_jobs=15)))


Xgboost F1-Micro:  0.7515780829697506 



Random forest gini(class balance=None) F1-Micro:  0.7395136626490305 



Random forest entropy F1-Micro:  0.7363018560941823 





In [224]:
# from pandas_profiling import ProfileReport
# profile = ProfileReport(train, minimal=True).to_notebook_iframe()
#
# profile

In [225]:
submission = sample_submission.copy()
submission['damage_grade'] = xgboost_test_pred
submission.to_csv('xgboost.csv', index=False)

In [226]:
# from sklearn.feature_selection import RFECV
#
# encoder = ce.OrdinalEncoder()
#
#
# x_train_encoded = encoder.fit_transform(X_train)
#
# model = XGBClassifier(seed=1337, n_jobs=15, colsample_bytree=0.2, gamma=0,
#                                                         learning_rate=0.09, max_depth=26, min_child_leaf=3, min_child_weight=4,
#                                                         n_estimators=260, scale_pos_weight=25, subsample=0.9)
#
#
#
# feature_selector = RFECV(model, n_jobs=15, cv=3, scoring='f1_micro', step=2)
# feature_selector.fit(x_train_encoded, y_train)
#
# features_mask = feature_selector.support_
# features_ranking = feature_selector.ranking_
#
# print(feature_selector.n_features_)

#print("New F1-Score: ", f1_score())

In [ ]:
import matplotlib.pyplot as plt

if tune:
    model = forest_search.best_estimator_.named_steps.model
    encoder = forest_search.best_estimator_.named_steps.encoder
else:
    model = xgboost_pipeline.named_steps.model
    encoder = xgboost_pipeline.named_steps.encoder


encoded_columns = encoder.transform(X_test).columns
importances = pd.Series(model.feature_importances_, encoded_columns)
plt.figure(figsize=(10,30))
importances.sort_values().plot.barh(color='grey');